# Master Data Science, Lyon 1
# TP2 PGM (durée 6h) 

## NGUYEN Xuan Huy - p1419771

Alex Aussem (alexandre.aussem@univ-lyon1.fr)

***

## Chatbot with a sequence to sequence network and attention
*************************************************************

In this tutrial, you will learn how to teach a recurrent neural network to anwers various questions about the admission to the master in Data Science of Lyon 1** to varying degrees of success... 

From Siri to Google Translate, deep neural networks have enabled breakthroughs in machine understanding of natural language. Most of these models treat language as a flat sequence of words or characters, and use a kind of model called a recurrent neural network (RNN) to process this sequence

The code was adapated by Alex Aussem from the Pytorch example "Translation with a Sequence to Sequence Network and Attention". **Author**: `Sean Robertson <https://github.com/spro/practical-pytorch>`

Sentences are in French. Example: [> input, = target, < output]

    
    > pourriez vous me renseigner ?
    = oui volontiers d ou venez vous ?
    < oui volontiers <EOS>

    > quel est le corps professoral ?
    = il est compose de chercheurs du liris 
    < il est compose de chercheurs du liris  <EOS>
    
To deal with these types of variable-length input and output, we need to use a recurrent neural network (RNN). Feed-forward neural networks, such as convolutional neural networks, do not maintain internal state other than the network’s own parameters. An RNN maintains its internal state while reading a sequence of inputs, which in our case will be a sequence of words, thereby being able to process an input of any length.

This is made possible by the simple but powerful idea of the `sequence
to sequence network <http://arxiv.org/abs/1409.3215>`, in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

To improve upon this model we'll use an **attention mechanism**, which lets the decoder
learn to focus over a specific range of the input sequence. The approach of incorporating attention mechanism has become one of the hottest topics in deep learning recently (<https://arxiv.org/abs/1507.01053>, <https://arxiv.org/abs/1409.0473>`).


**Recommended Reading:**

-  http://pytorch.org/ For installation instructions
-  doc:`/beginner/deep_learning_60min_blitz` to get started with PyTorch in general
-  doc:`/beginner/pytorch_with_examples` for a wide and deep overview


It would also be useful to know about Sequence to Sequence networks and
how they work:

-  'Recursive Neural Networks with PyTorch with a recurrent tracker and TreeLSTM nodes
    <https://devblogs.nvidia.com/parallelforall/recursive-neural-networks-pytorch>`
-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Loading data files
==================

The data for this project is a set of questions/anwers pairs.


We will be representing each word in a language as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word). Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
hundred words per language.

We'll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called ``Lang`` which has word → index (``word2index``) and index → word
(``index2word``) dictionaries, as well as a count of each word
``word2count`` to use to later replace rare words.




In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split():
            if word == '':
                print('****************',sentence)
            self.addWord(word)
         
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters except digits
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    #s = re.sub(r"[^a-zA-Z0-9?]+", r" ", s)
    #s = re.sub(r"[^a-zA-Z0-9?&\'\’\%\-]+", r" ", s)
    s = re.sub(r"[^a-zA-Z0-9?&\%\-]+", r" ", s)
    return s

In [ ]:
s="Il eût été bien de s'inscrire en 2017 !! N'est-ce pas ?"
s = normalizeString(s)
print(s)

To read the data file we will split the file into lines, and then split
lines into pairs. If we want to infer the question from the answers, the ``reverse``
flag is added to reverse the pairs.


In [ ]:
def readLangs(questions, answers, reverse=False):
    print("Reading lines...")
        
    lines = open('data/chatbot-M2-DS.txt', encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(answers)
        output_lang = Lang(questions)
    else:
        input_lang = Lang(questions)
        output_lang = Lang(answers)

    return input_lang, output_lang, pairs

Here we set a limit to the length of input words (that includes ending punctuation + EOS token). The maximum input sentence length has to be determined in advance in order to parametrize the attention mechanism.

In [ ]:
MAX_LENGTH = 15

#stopwords = 'de du des d le la les l ce c ci ça m me ma si t sur n en s si a y au un une on il nous vous je j'.split()

stopwords = []

def TrimWordsSentence(sentence):
    resultwords = [word for word in sentence.split() if word.lower() not in stopwords]
    resultwords = ' '.join(resultwords)
    return resultwords

def TrimWords(pairs):
    for pair in pairs: 
        pair[0] = TrimWordsSentence(pair[0])
        #pair[1] = pair[1].strip()
    return pairs

def filterPair(p):
    return len(p[0].split()) < MAX_LENGTH and \
        len(p[1].split()) < MAX_LENGTH 

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
s="Il eût été bien de s'inscrire en 2017 !! N'est-ce pas ?"
print(s)
s = normalizeString(s)
print(s)
s = TrimWordsSentence(s)
print(s)

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = TrimWords(pairs)
    
    for pair in [pair for pair in pairs if not filterPair(pair)]:
        print('%s (%d) -> %s (%d)' % (pair[0],len(pair[0].split()),pair[1],len(pair[1].split())))  
    
    pairs = filterPairs(pairs)
    
    print('')
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        if '' in output_lang.word2index: print(pair[1].split())
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [ ]:
input_lang, output_lang, pairs = prepareData('questions', 'answers', False)
print(random.choice(pairs))
#print(indexesFromSentence(input_lang,'bonjour'))
#print(input_lang.index2word[102], '->',input_lang.word2index['bonjour'])

In [ ]:
print(set(input_lang.word2index))

In [ ]:
print(set(output_lang.word2index))

In [ ]:
pairs

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for dialogue systems.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        return result

A standard RNN Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to produce the answer.




**Simple Decoder**

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).



In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]),dim=1)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        return result

# Attention Decoder


If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.



In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)))#, dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]))#, dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        return result

Note: There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`

# Training


Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split()]

def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    return result

def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct answer -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the answer
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [ ]:
teacher_forcing_ratio = 0.5

def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            #decoder_output, decoder_hidden = decoder(
             #   decoder_input, decoder_hidden)
            
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            #decoder_output, decoder_hidden = decoder(
             #   decoder_input, decoder_hidden)       
            
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0].item()

            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input

            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    training_pairs = [variablesFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]

        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    input_variable = variableFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))  # SOS
    decoder_input = decoder_input

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
        
        #decoder_output, decoder_hidden = decoder(
         #   decoder_input, decoder_hidden)

        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        decoder_attentions[di] = decoder_attention.data

        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0].item()

        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])

        decoder_input = Variable(torch.LongTensor([[ni]]))

    return decoded_words, decoder_attentions[:di + 1]
    #return decoded_words

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        #output_words = evaluate(encoder, decoder, pair[0])
     
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it's easier to run multiple experiments easier) we can actually
initialize a network and start training.

Remember that they are very few input sentences. For this small
dataset we can use relatively small networks of 100 hidden nodes and a
single GRU layer. After about 3 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [ ]:
hidden_size = 256
#hidden_size = 100
encoder1 = EncoderRNN(input_lang.n_words, hidden_size)

attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,
                               1, dropout_p=0.1)
#attn_decoder1 = DecoderRNN(hidden_size, output_lang.n_words,1)

#trainIters(encoder1, attn_decoder1, 75000, print_every=5000)
trainIters(encoder1, attn_decoder1, 10000, print_every=1000)

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

New questions
=========

Ask new questions (never seen during training) using only the words that appear in your input vocabulary. 

In [ ]:
mail="Bonjour, je suis intéressé par la data science. C'est quand la rentrée ?\
Quels sont les cours enseignés  ? Quelle est la durée du stage ?\
On peut faire un stage recherche en laboratoire ? Merci monsieur."

sentences = [s.strip() for s in re.split('[\.\,\?\!]' , mail)]
print(sentences[:-1])
sentences = sentences[:-1]

for sentence in sentences : #sentence.split() :
    sentence2= TrimWordsSentence(normalizeString(sentence))
    #print(sentence2)
    output_words, attentions = evaluate(encoder1, attn_decoder1, sentence2)
    output_sentence = ' '.join(output_words)
    print(sentence)
    #print('=', sentence2)
    print('->', output_sentence)
    print('')
    

Visualizing Attention
---------------------

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

We simply run ``plt.matshow(attentions)`` to see attention output
displayed as a matrix, with the columns being input steps and rows being
output steps:

For a better viewing experience we will do the extra work of adding axes
and labels:




In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    showAttention(input_sentence, output_words, attentions)

evaluateAndShowAttention(TrimWordsSentence("bonjour"))
evaluateAndShowAttention(TrimWordsSentence("quelle est la date de la rentree 2018 ?"))
evaluateAndShowAttention(TrimWordsSentence("il faut faire un stage en entreprise  ?"))

---
# TO DO



-  Discuss the weaknesses of this simple approach
-  Allow words in the new questions that were unseen during training.
-  Allow questions of > 10 words (for instance, by selecting 10 words at random in the sentence).
-  Adapt the code to handle several sequences at once (like an email)
-  Replace the embeddings with pre-trained word embeddings such as word2vec (or GloVe).
-  Try with more layers, more/less hidden units, and more sentences. Compare
   the training time and performance.
-  Define and plot other loss functions for this specific task (e.g. % correct words)


# Further ideas (optional)
- Use "negative sampling", i.e. sampling random words to form incorrect questions (answering the question with something like 'I don't understand your question, sorry.' in order to enforce the model recognize a questions that are not syntactically (and semantically) correct. 
The probability for selecting a word as a negative sample should be related to its frequency, with more frequent words being more likely to be selected as negative samples. 
- To reduce the size of the input vocabulary and thus the computational burden, try to replace all the unknwon words with an unknown token <UNK> instead of ignoring them.
-  Adapt the decoder to infer directly the word embeddings (semantic) in the output sentence.  



#### Discuss the weaknesses of this simple approach

Cet approche est très sensible (les fautes orthographes, les nouveaux vocabulaires, les mots manquants) et c'est facile de planter le chatbot just par dupliquer le même mot plusieurs fois
Il serait intéressant d'implémenter un fonction du traitement bidirectionnel au lieu d'un réseau de neurones unidirectionnel. Et on peut ignorer les mots rares.

#### Allow words in the new questions that were unseen during training

In [ ]:
def indexesFromSentence(lang, sentence):
    indexes=[]
    for word in sentence.split(' '):
        if word in set(lang.word2index):
            indexes.append(lang.word2index[word])   
        else:
            print("Unknwon word : ",word)
    while len(indexes) >= MAX_LENGTH:
        indexes.pop(random.randint(1,len(indexes)-1))
    return indexes

In [ ]:
new_sentence = "c'est NGUYEN Xuan Huy p1419771 ?"
indexes = indexesFromSentence(input_lang, TrimWordsSentence(normalizeString(new_sentence)))
print(indexes)

In [ ]:
def respond(message):
    sentence = normalizeString(message)
    output_words, attentions = evaluate(encoder1, attn_decoder1, sentence)
    output_sentence = ' '.join(output_words)
    print(sentence)
    print('->', output_sentence)

In [ ]:
respond('c\'est NGUYEN Xuan Huy p1419771 ?')

In [ ]:
def SentenceFromIndexes(lang, indexes):
    sentence = []
    for i in indexes:
        sentence.append(input_lang.index2word[i])
    sentence = ' '.join(sentence)
    return sentence

In [ ]:
new_sentence = "pourriez vous me dire quels sont les professeurs ou les \
PhD du UCBL qui vont encadrer mon stage de fin \
d'étude s'il vous plaît ?"
indexes = indexesFromSentence(input_lang, TrimWordsSentence(normalizeString(new_sentence)))
sentence = SentenceFromIndexes(input_lang, indexes)
print(new_sentence)
print('->', sentence)
evaluateAndShowAttention(sentence)

#### Allow questions of > 10 words (for instance, by selecting 10 words at random in the sentence)

On va adjuster un peu dans le fonction evaluate()

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    input_variable = variableFromSentence(input_lang, sentence)
    
    if len(input_variable) >= max_length:
        selection = random.sample(range(len(input_variable)), max_length)
        input_variable = input_variable[sorted(selection)]
        
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))  # SOS
    decoder_input = decoder_input

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
        
        #decoder_output, decoder_hidden = decoder(
         #   decoder_input, decoder_hidden)

        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        decoder_attentions[di] = decoder_attention.data

        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0].item()

        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])

        decoder_input = Variable(torch.LongTensor([[ni]]))

    return decoded_words, decoder_attentions[:di + 1]
    #return decoded_words

In [ ]:
message = 'Le réseau Long Short Term Memory et RNN sont-ils abordés dans le parcours Data Science ? ? ? ? ? ? ? ? ?'
respond(message)
respond(message)
respond(message)
respond(message)
respond(message)

#### Adapt the code to handle several sequences at once (like an email)

In [ ]:
def mail_reply(mail):
    sentences = [s.strip() for s in re.split('[\.\,\?\!]' , mail)]
    #print(sentences[:-1])
    sentences = sentences[:-1]

    for sentence in sentences : #sentence.split() :
        sentence2= TrimWordsSentence(normalizeString(sentence))
        #print(sentence2)
        output_words, attentions = evaluate(encoder1, attn_decoder1, sentence2)
        output_sentence = ' '.join(output_words)
        print(sentence)
        #print('=', sentence2)
        print('->', output_sentence)
        print('')

In [ ]:
mail="Bonjour, je suis attiré par le data science à UCBL. C'est quand la rentrée ?\
Quels sont les cours enseignés ? Quelle est la durée du stage, 6 ou 5 mois ?\
Qui est le responsabilite ?\
Peut-on faire un stage recherche en entreprise ? Merci madame, monsieur."

mail_reply(mail)

#### Replace the embeddings with pre-trained word embeddings such as word2vec (or GloVe).

In [ ]:
from gensim.models import KeyedVectors
filename = 'data/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
words = list(model.vocab)
print(set(input_lang.word2index).intersection(words))
l1 = len(list(set(input_lang.word2index).intersection(words)))
print('Percentage of common words w.r.t. input_lang : %0.2f%%' % (100 * l1/input_lang.n_words))
print('Percentage of common words w.r.t. word2vec vocab : %0.2f%%' % (100 * l1/len(words)))

In [ ]:
print(words[11000:11200])

In [ ]:
def indexesFromSentence(lang, sentence):
    indexes=[]
    for word in sentence.split():
        if word in set(lang.word2index):
            indexes.append(lang.word2index[word])
        elif word in model.vocab:
            new_word = model.most_similar(positive=[word], topn=1)[0][0]
            if new_word in set(lang.word2index):
                indexes.append(lang.word2index[new_word])
                print(word,'->',new_word)
        else:
            print("Unknwon word in model vocab: ",word)
    while len(indexes) >= MAX_LENGTH:
        indexes.pop(random.randint(1,len(indexes)-2))
    return indexes

In [ ]:
new_sentence = "Je suis viff, auriez-vous l'amabiliiiité de me dire quels sont les enseignants ou les chercheurs \
du laboratoire qui vont encadrer mon stage de fin d'étude cette année et qui est le responsable du master s'il vous plaît ?"
trimmed_sentence = TrimWordsSentence(normalizeString(new_sentence))
indexes = indexesFromSentence(input_lang, trimmed_sentence)
print(indexes)
print(new_sentence)
print('->', trimmed_sentence)
print('->', SentenceFromIndexes(input_lang, indexes))

In [ ]:
vocab_dim = 200
n_symbols = input_lang.n_words + 1

embedding_weights = np.zeros((n_symbols, vocab_dim))
for word,index in input_lang.word2index.items():
    try:
        embedding_weights[index, :] = model.get_vector(word)
    except KeyError:
        pass

print(embedding_weights.shape)
print(len(model.vocab))

In [ ]:
hidden_size = vocab_dim

class EncoderRNN_with_trained_embedding(nn.Module):
    def __init__(self, input_size, n_symbols, pretrained_embed, n_layers=1):
        super(EncoderRNN_with_trained_embedding, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        pretrained_embed = torch.Tensor(pretrained_embed)
        self.embedding.weight = nn.Parameter(pretrained_embed)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        return result

class AttnDecoderRNN_with_trained_embedding(nn.Module):
    def __init__(self, hidden_size, output_size, pretrained_embed, n_layers=1, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN_with_trained_embedding, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        pretrained_embed = torch.Tensor(pretrained_embed)
        self.embedding.weight = nn.Parameter(pretrained_embed)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)))#, dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]))
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        return result

In [ ]:
encoder1 = EncoderRNN_with_trained_embedding(input_lang.n_words, hidden_size, embedding_weights)
attn_decoder1 = AttnDecoderRNN_with_trained_embedding(hidden_size, output_lang.n_words,
                                                      embedding_weights,1, dropout_p=0.1)

In [ ]:
trainIters(encoder1, attn_decoder1, 5000, print_every=100)

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

In [ ]:
new_sentence = "Bonjour, je suis en intérim, je vis sur un promontoire, je porte un crucifix autour du cou, suis-je en mesure de faire un stage de fin d'étude sans souci ?"

trimmed_sentence = TrimWordsSentence(normalizeString(new_sentence))
indexes = indexesFromSentence(input_lang, trimmed_sentence)
print(indexes)
print(new_sentence)
print('->', trimmed_sentence)
final_sentence = SentenceFromIndexes(input_lang, indexes)
print('->', final_sentence)
evaluateAndShowAttention(final_sentence)

#### Try with more layers, more/less hidden units, and more sentences. Compare the training time and performance.

#### Define and plot other loss functions for this specific task (e.g. % correct words)

## Further ideas

In [ ]:
import spacy
nlp_fr = spacy.load('fr_core_news_md')

In [ ]:
mail = "Bonjour Monsieur Aussem, je suis titulaire d'un Master 2 mention informatique et actuellement.\
Je souhaite savoir si mes deux ans de doctorat pourront constituer un obstacle lors de ma demande \
de candidature en Master 2 mention informatique parcours \
Data Science au sein de votre université. Cordialement."

text_fr = nlp_fr(unidecode(mail))

for token in text_fr:
    print(token.text, token.lemma_, '\t', token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

In [ ]:
token1 = nlp_fr(unidecode('bonjour'))
token2 = nlp_fr(unidecode('monsieur'))

print(token1.similarity(token2))
print(token1.vector)

In [ ]:
text = []
stop_tags = {'PUNC','ADP_','DET_'}
for w in text_fr:
    if w.tag_[0:4] not in stop_tags:
        print("%s\t%s\t%s" % (w, w.tag_[0:4], w.dep_))
        text.append(w.text)
text2 = ' '.join(text)
print(text2)

In [ ]:
doc_fr = text_fr
for sent in doc_fr.sents:
    print(sent)

In [ ]:
words = [w.text for w in doc_fr]
print(words)

In [ ]:
import unidecode
from unidecode import unidecode

unidecode("Il eût été bien de s'inscrire en 2017 !! N'est-ce pas ?")

In [ ]:
import nltk

In [ ]:
tokens = nltk.word_tokenize(unidecode(text_fr))
print(tokens)

In [ ]:
tagged = nltk.pos_tag(tokens)
print(tagged)

In [ ]:
entities = nltk.chunk.ne_chunk(tagged)
entities

In [ ]:
import gensim
message = gensim.utils.to_unicode(text_fr, 'latin1').strip()
print(message)
message = list(gensim.utils.tokenize(message, lower=True))
print(message)
text = ' '.join(message)
print(text)